# Run memento for Perturb-seq benchmarking


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2


In [3]:
import sys
sys.path.append('/home/ssm-user/Github/memento')

In [4]:
import scanpy as sc
import scipy.stats as stats
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import string
import random
import logging
import itertools
import statsmodels.api as sm
import seaborn as sns
import random

In [5]:
import memento

In [6]:
import memento.model.rna as rna
import memento.estimator.hypergeometric as hg

In [7]:
data_path = '/data_volume/memento/'

In [8]:
logging.basicConfig(
    format="%(asctime)s %(process)-7s %(levelname)-8s %(message)s",
    level=logging.INFO, 
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.captureWarnings(True)

### Perturbseq

In [9]:
adata = sc.read(data_path + 'tfko140/tfko.sng.guides.full.ct.details.h5ad')

Only considering the two last: ['.details', '.h5ad'].
Only considering the two last: ['.details', '.h5ad'].


In [10]:
donors = adata.obs.donor.drop_duplicates().tolist()

In [11]:
guide_pairs = pd.read_csv(data_path + 'tfko140/benchmarking/guide_pairs_to_test.csv')

In [12]:
rna.MementoRNA.setup_anndata(
    adata=adata,
    q_column='q',
    label_columns=['donor', 'guide'])

2023-06-29 18:24:43 3935896 INFO     setup_anndata: creating groups
2023-06-29 18:25:06 3935896 INFO     setup_anndata: computing cell sizes


In [13]:
# Define expressed genes
adata.var['expr_genes'] = (adata.X.mean(axis=0).A1 > 0.01)
adata = adata[:, adata.var['expr_genes']]

In [25]:
model = rna.MementoRNA(adata=adata)

In [26]:
model.compute_estimate(
    estimator=[
        'mean', 
        'sem', 
        'var', 
        'total_umi',
        'cell_count'])

2023-06-29 18:39:33 3935896 INFO     compute_estimate: gene_list is None, using all genes in AnnData object...


In [ ]:
for idx, row in guide_pairs.iterrows():
    
    sets = [(row['target1_guide1'], row['target2_guide1']), (row['target1_guide2'], row['target2_guide2'])]
    for t1, t2 in sets:
        

        groups = [f'memento_group^{d}^{g}' for d,g in itertools.product(donors, [t1,t2])]
        treatment = pd.DataFrame(index=groups)
        covariate = pd.DataFrame(index=groups)
        treatment['is_target1'] = treatment.index.str.contains(t1).astype(float)
        covariate['donor'] = covariate.index.str.split('^').str[1]
        covariate = pd.get_dummies(covariate, drop_first=True).astype(float)
        covariate = sm.add_constant(covariate)
        
        result = model.differential_mean(covariates=covariate, treatments=treatment, family='NB', n_jobs=5)
        result.to_csv(data_path + f'tfko140/benchmarking/memento_results/{t1}_{t2}.csv')

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/varfuncs.py:265: RuntimeWarning: overflow encountered in square
  return p + self.alpha*p**2
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:132: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:516: RuntimeWarning: overflow encountered in exp
  return np.exp(z)
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1342: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/ssm-user/anaconda3/envs/single_cell/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1346: RuntimeWarning: divide by zero encountered in log
  resid_dev -= endog_alpha * np.log